In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers

In [2]:
# Define constants
IMG_SIZE = 465  # Target image size (465x465)
DATA_DIR = "C:/Users/samik/Downloads/NewOriginalSlices-20250318T213744Z-001/NewOriginalSlices"  # Replace with your actual data path
CATEGORIES = ["0", "1"]

In [3]:
# Best hyperparameters from tuning
best_params = {
    "conv_1_filter": 64,       
    "conv_1_kernel": 5,
    "conv_2_filter": 64,
    "conv_2_kernel": 3,
    "dense_units": 128,
    "dropout": 0.2,
    "learning_rate": 0.00013414267455355165
}

In [4]:
def load_and_preprocess_images():
    images, labels = [], []
    
    for label, category in enumerate(CATEGORIES):
        folder_path = os.path.join(DATA_DIR, category)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            
            if img is None:
                continue  # Skip if image is not loaded properly
            
            h, w = img.shape

            # Compute padding values
            top_pad = (IMG_SIZE - h) // 2
            bottom_pad = IMG_SIZE - h - top_pad
            left_pad = (IMG_SIZE - w) // 2
            right_pad = IMG_SIZE - w - left_pad

            # Apply zero padding to reach 465x465
            padded_img = np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad)), mode='constant', constant_values=0)
            
            images.append(padded_img)
            labels.append(label)

    images = np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # Reshape for CNN
    labels = np.array(labels)

    return images, labels

In [5]:
# Load and preprocess images
X, y = load_and_preprocess_images()

# Split into training (80%) and validation (20%) sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalize pixel values
X_train, X_val = X_train / 255.0, X_val / 255.0

In [6]:
# Build CNN Model using best hyperparameters
model = models.Sequential([
    layers.Conv2D(best_params["conv_1_filter"], (best_params["conv_1_kernel"], best_params["conv_1_kernel"]), 
                  activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(best_params["conv_2_filter"], (best_params["conv_2_kernel"], best_params["conv_2_kernel"]), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(best_params["dense_units"], activation='relu'),
    layers.Dropout(best_params["dropout"]),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

C:\Users\samik\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model using the best learning rate
optimizer = optimizers.Adam(learning_rate=best_params["learning_rate"])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

Epoch 1/10
14/37 ━━━━━━━━━━━━━━━━━━━━ 2:50 7s/step - accuracy: 0.5963 - loss: 0.7266

KeyboardInterrupt: 